In [14]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
import pickle
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

epochs = 40
batch_size = 32

image_gen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split=0.2,
                    )
train_dir = os.path.join(os.getcwd(), 'train_dataset')

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle= True,
                                                     class_mode='sparse',
                                                     color_mode='grayscale',
                                                     subset='training',
                                                     target_size = (64, 64))
print(train_data_gen.class_indices)
val_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                                directory = train_dir,
                                                shuffle=True,
                                                class_mode='sparse',
                                                color_mode='grayscale',
                                                subset='validation',
                                                target_size=(64,64))

                                        



Found 18480 images belonging to 2 classes.
{'original': 0, 'upscaled': 1}
Found 4620 images belonging to 2 classes.


In [15]:
model = Sequential() 

model.add(Conv2D(16, (3,3), input_shape = (64, 64, 1)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))
model.add(Conv2D(16, (3,3)))

model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(MaxPooling2D(pool_size=(4,4)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(2))
model.add(Activation("softmax"))

# Compiling the model using some basic parameters
model.compile(loss="sparse_categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"])


# model.summary()

In [16]:
import keras
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(os.getcwd(), 'models/new_detect_upscale_model_{epoch}.h5'),
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1),
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor='val_loss',
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 3 epochs"
        patience=4,
        verbose=1)
]

history = model.fit(train_data_gen, validation_data=val_data_gen, epochs=epochs,
                    steps_per_epoch = train_data_gen.samples // batch_size,
                    validation_steps = val_data_gen.samples // batch_size,
                    callbacks=callbacks)


Train for 577 steps, validate for 144 steps
Epoch 1/40
576/577 [============================>.] - ETA: 0s - loss: 0.6964 - accuracy: 0.5456
Epoch 00001: val_loss improved from inf to 0.69946, saving model to /Users/jiawei/CS386/project3/models/new_detect_upscale_model_1.h5
577/577 [==============================] - 403s 698ms/step - loss: 0.6964 - accuracy: 0.5456 - val_loss: 0.6995 - val_accuracy: 0.5584
Epoch 2/40
576/577 [============================>.] - ETA: 0s - loss: 0.6799 - accuracy: 0.5824
Epoch 00002: val_loss improved from 0.69946 to 0.65965, saving model to /Users/jiawei/CS386/project3/models/new_detect_upscale_model_2.h5
577/577 [==============================] - 449s 779ms/step - loss: 0.6799 - accuracy: 0.5824 - val_loss: 0.6596 - val_accuracy: 0.5801
Epoch 3/40
576/577 [============================>.] - ETA: 0s - loss: 0.3683 - accuracy: 0.8282
Epoch 00003: val_loss improved from 0.65965 to 0.09785, saving model to /Users/jiawei/CS386/project3/models/new_detect_upscale

In [24]:
model = tf.keras.models.load_model( os.getcwd() + '/models/new_detect_upscale_model_7.h5')
''' 64x64 patch taken from the centre of all 200 4K images; 
    200 64x64 patch of Bicubic upscaled from 1080p
    and 200 64x64 patch of Lanczos upscaled from 1080p '''
test_gen = ImageDataGenerator(rescale=1./255)
test_dir = os.path.join(os.getcwd(), 'test_dataset')
test_data_gen = test_gen.flow_from_directory(batch_size=batch_size,
                                                     directory=test_dir,
                                                     shuffle= True,
                                                     class_mode='sparse',
                                                     color_mode='grayscale',
                                                     target_size = (64, 64))
print(test_data_gen.class_indices)



Found 600 images belonging to 2 classes.
{'original': 0, 'upscaled': 1}


In [25]:
print("=============== Train dataset metrics ===============")
print(model.evaluate(train_data_gen))
print("=============== Validation dataset metrics ===============")
print(model.evaluate(val_data_gen))
print("=============== Test dataset metrics ===============")
print(model.evaluate(test_data_gen))


=============== Train dataset metrics ===============
578/578 [==============================] - 69s 120ms/step - loss: 0.1107 - accuracy: 0.9601
[0.11066196460895575, 0.96006495]
=============== Validation dataset metrics ===============
145/145 [==============================] - 16s 109ms/step - loss: 0.0343 - accuracy: 0.9924
[0.03434816428684983, 0.99242425]
=============== Test dataset metrics ===============
19/19 [==============================] - 2s 118ms/step - loss: 0.1039 - accuracy: 0.9633
[0.1039439278997873, 0.9633333]


In [19]:
model.predict(test_data_gen)

array([[0.04575605, 0.95424396],
       [0.07001948, 0.92998046],
       [0.94701076, 0.05298924],
       ...,
       [0.99824214, 0.00175788],
       [0.0092172 , 0.9907828 ],
       [0.03011798, 0.9698821 ]], dtype=float32)